# Writing Views, Mapping URLs and Accepting URL Arguments

In the previous notebook, you learned how to make and use models. In this notebook, we'll create controllers, use the model in our controller, and map the controller to a URL. We'll also learn a few other things along the way.

## Basic Function-Based View

In Django, controllers are written in __`views.py`__. It contains the logic needed to respond to requests. Let's write a basic function.

In [ ]:
# In account/views.py
from django.http import HttpResponse

def basic_view(request):
    return HttpResponse("Hello World")

A very simple view can look like this. It just returns a string. `HttpResponse` is a low level response class, meaning there are other functions that can build on it. We'll use something better later.

## App URLs

In order to map our view function to an address, we add it to our __app's `urls.py`__.

In [ ]:
# In account/urls.py  <- this is our app
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^$', views.basic_view),  # r = raw string, ^ = regex for beginning of string, $ = regex for end of string
]

From the same package, `views.py` is imported. It is passed as the second argument to the `url()` function. The `urlpatterns` variable is a set of URL patterns. The minimum requirement for URLs are: a regular expression and a view function or class, as a view. Class-based views will be explained later. For now, let's focus on writing function-based views. The regular expression is used to match addresses and the function determines the logic related to that address.

When a URL is requested, Django searches matching url patterns and uses the first match so it's important to use unique URL namespaces and patterns to prevent any conflicts. Because of its pattern, our URL can be found in the root of its namespace. In our example, the pattern is an empty string. It would be good to get a regular expression cheatsheet or read up on basics of regular expressions if you didn't understand it.

## Project URLs

Our function is now within the app URLs, but our app's urls aren't connected to our project yet. We also have to include our app's urls into the __project's `urls.py`__ in the project package folder (the one with `settings.py` and `wsgi.py`).

In [ ]:
# In google/urls.py  <- this is our project
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^account/', include('account.urls'))
]

The URLs of our `account` app has been added to the `account/` namespace in our app. When a user navigates to this url, Django searches URL patterns within the included app. Our empty string pattern matches the root of that address so navigating to the address `account/` calls our function.

If no server is running, use the `python manage.py runserver` command to have one running so we can see the results. In your browser, navigate to [127.0.0.1:8000/account/](http://127.0.0.1:8000/account/) to see the result.


![Web_Dev2](./images/img_hello_world.png)


Congratulations! You made Django work!

## [Render](https://docs.djangoproject.com/en/dev/topics/http/shortcuts/#render)

We can show text but that's that's quite boring. Let's show some real HTML! Let's create a new function that does that. There are many ways to do this in Django. Let's try the commonly used `render()` function.

In [ ]:
# In account/views.py
from django.shortcuts import render

def render_view(request):
    template = "template.html"
    context = {"variable": "my_context_variable"}
    return render(request, template, context)

The `render()` function accepts the request object, a template string and a context dictionary. Not only can be show any template we want, we can add any variable into the page's context.

## App URLs

Now let's add this view to our app's URLs.

In [ ]:
# In account/urls.py
from django.conf.urls import patterns, url

from . import views

urlpatterns = [
    url(r'^$', views.basic_view),
    url(r'^render/$', views.render_view),
]

Remember that this app lives under the `account/` URL. Our second function can be reached if a user navigates to `account/render/`. But we don't have a template yet so let's create one.

## Template 

Our new view adds a variable in the page context. Not only will we make an HTML page appear, we'll be able to add variables to it. Let's have a simple template showing our variable. We'll get more into templates in the next notebook. For now, it's good enough that we can show variables sent to the page's context.

#### Note: The code below is just an example and you may run it here on the Jupyter notebook.

In [ ]:
%%html
<html>
<head>
  <title>Hello world</title>
</head>
<body>

<h1>Hello world</h1>

<p>{{ variable }}</p>
<pre>"variable" is a key in our context dictionary</pre>
</body>
</html>

There's something different in our template - the double curly braces around the word `variable`. Remember that our context is a dictionary and `variable` is one of its keys. The double curly braces mean that we are accessing a variable that is a key in our page's context. This is how variables can be shown in a template. We'll learn more about templates in a nother notebook.

Since we've already added this app's URLs to our project, this view will already be accessible in the assigned namespace. Notice the regular expression pattern. You know what that means, right? Visit [127.0.0.1:8000/account/render/](http://127.0.0.1:8000/account/render/) to see the results.

## [Redirect](https://docs.djangoproject.com/en/dev/topics/http/shortcuts/#redirect)

Sometimes, we don't want a URL to render a page, but to redirect to another URL. And, you guessed it, there's a function for that called `redirect()`.

`redirect()` will work with several arguments:

* an object, its __[`get_absolute_url()`](https://docs.djangoproject.com/en/dev/ref/models/instances/#django.db.models.Model.get_absolute_url)__ method will be called.
* a hardcoded relative or absolute URL
* the dotted Python path to the view function as a string
* the name of a URL pattern (which will be looked up using the [`reverse()`](https://docs.djangoproject.com/en/dev/ref/urlresolvers/#django.core.urlresolvers.reverse) function)

## Named URL patterns

In the previous section, named URL patterns were mentioned. This is the recommended way to refer to URLs so we won't try the other methods here (Feel free to research them on your own though). Using this method, actual URL patterns or dotted Python paths can change, yet, referrences need __NOT__ be changed. References could be scattered all across your project in various controllers or templates. Referring to names allow you to be flexible about the implementation without having to change URL references all over the place all the time, whenever a detail changes.

It's simple to name a URL. Just add a keyword argument `name=URL_name` with `URL_name` as your name. Let's give our URLs some names.

In [ ]:
# In account/urls.py
urlpatterns = [
    url(r'^$', views.basic_view, name="hello_world"),
    url(r'^render/$', views.render_view, name="render"),
    url(r'^redirect/$', views.redirect_view, name="redirect"),
]

Now we can refer to them with their names.

In [ ]:
# In account/views.py
from django.shortcuts import redirect

def redirect_view(request):
    return redirect("render")  # try changing this to any of the other URL names

As mentioned, `redirect()` uses [`reverse()`](https://docs.djangoproject.com/en/dev/ref/urlresolvers/#django.core.urlresolvers.reverse) to find URLs. It goes without saying that this also works in the `reverse()` and [`reverse_lazy()`](https://docs.djangoproject.com/en/dev/ref/urlresolvers/#reverse-lazy) functions. Reverse here doesn't mean the sort order of something will be changed, it simply means to do a reverse lookup.

## Namespaced URLs

Now that we can name URL patterns, we may have the problem of our namespace being possibly too crowded when our app grows. Let's say our project has several apps and they all involve Creating, Reading, Updating, and Deleting a model. We can give them all unique names and that would work. We'll have the complete set of `account_create`, `dashboard_read`, `profile_update`, `settings_delete`, etc. for each model. It's simple enough and it will work. Or we can use namespaces.

In [ ]:
# In account/urls.py --> google?
urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^account/', include('account.urls', namespace='account'))
]

We passed an additional keyword argument to the `include()` function when we imported our app's URLs to our project. With this change, we can refer to our URL names using the format `namespace:name`. Each of our app namespaces are now isolated and clean.

## Models

We learned about models in the previous notebook. Let's use those models and see how to show a model in our page.

In [ ]:
# In account/views.py
from django.shortcuts import render

from .models import Profile

def model_view(request):
    template = "template.html"
    context = {
        "variable": "my_context_variable",
        "object": Profile.objects.get(id=1)
    }
    return render(request, template, context)

It's as simple as adding the object to the context and accessing its key in the template. In our case we added an key in our context identified as `object` so we add a variable called {{ object }} in our template. It's that simple! Now, let's put all our objects into our page like a newsfeed of profiles signing up to our site.

In [ ]:
# In account/views.py
from .models import Profile

def model_view(request):
    template = "template.html"
    context = {
        "variable": "my_context_variable",
        "object_list": Profile.objects.all().order_by('-created')
    }
    return render(request, template, context)

The `QuerySet` method `order_by()` was used to sort the objects according to the `created` field, and the `-` reverses it so it's in descending order with the latest ones at the top.

## [get_object_or_404](https://docs.djangoproject.com/en/dev/topics/http/shortcuts/#get-object-or-404)

Django provides some shortcuts for working with single objects and lists of objects. Instead of writing the boilerplate code for retrieving an object or returning a HTTP 404 response `get_object_or_404()` can be used.

In [ ]:
# In account/views.py
from django.http import Http404

from .models import Profile

def get_object(request):
    try:
        context = {
            "object": Profile.objects.get(pk=1),
        }
    except Profile.DoesNotExist:
        raise Http404("No Profile matches the given query.")

    return render(request, template, context)

The boilerplate code above can be replaced with a simple line of code. When an object is not found, an error page is instead returned.

In [ ]:
# In account/views.py
def get_object_or_404_view(request):
    context = {
        "object": get_object_or_404(Profile, pk=1),
    }
    return render(request, template, context)

The `try... except` block is handled by `get_object_or_404()`. Try changing the `pk` value to one that doesn't exist to see the error page.

## [Custom 404 Page](https://docs.djangoproject.com/en/dev/topics/http/views/#the-http404-exception)

Right now, we're seeing one of Django's debug pages. To show a custome 404 error page, we'll follow the instruction on the page. In `settings.py`, change the value of __`DEBUG`__ to __`False`__. When the server restarts and we visit the page again, we'll see an error printed in the console:

`CommandError: You must set settings.ALLOWED_HOSTS if DEBUG is False.`

Right now, __`ALLOWED_HOSTS`__ is a empty list. Change its value to __`['localhost']`__.

Create a __`404.html`__ page. This page will be shown on any HTTP 404 errors.

## [Passing Arguments in the URL](https://docs.djangoproject.com/en/dev/topics/http/urls/#example)

To capture a value from a URL, just put parentheses around it. Here are some examples:

In [ ]:
# In account/urls.py
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^articles/2003/$', views.special_case_2003),
    url(r'^articles/([0-9]{4})/$', views.year_archive),
    url(r'^articles/([0-9]{4})/([0-9]{2})/$', views.month_archive),
    url(r'^articles/([0-9]{4})/([0-9]{2})/([0-9]+)/$', views.article_detail),
]

Browsing to /articles/2005/03/ would match the third pattern in the list. With `2005` and `03` matching the patterns in the parenthesis, they are passed as positional arguments. Django calls `views.month_archive(request, '2005', '03')`. The pattern requires four digits then two digits so it can't match anything with one or three digits in either pattern.

Browsing to /articles/2003/ matches the first pattern because Django searches the list in order. If the trailing slash is missing, is also can't match since our pattern requires it.

To pass a keyword argument to a URL, named regular-expression groups are used. The pattern is `(?P<name>pattern)`, where `name` is the group name and pattern is the regular expression pattern. It's enclosed in a parenthesis so the value matching this pattern will be the value to the name as a keyword argument passed to the function.

In [ ]:
# In account/urls.py
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^articles/2003/$', views.special_case_2003),
    url(r'^articles/(?P<year>[0-9]{4})/$', views.year_archive),
    url(r'^articles/(?P<year>[0-9]{4})/(?P<month>[0-9]{2})/$', views.month_archive),
    url(r'^articles/(?P<year>[0-9]{4})/(?P<month>[0-9]{2})/(?P<day>[0-9]{2})/$', views.article_detail),
]

Browsing to /articles/2005/03/ would match calls `views.month_archive(request, year='2005', month='03')`. Similar to our unnamed pattern earlier but now returns keyword arguments because of the named regular expression groups.

As you may have noticed, the year and month arguments were passed as strings, not `int`. All arguments are captured from the URL are strings.